In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np
import pandas as pd
# Plot Images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
plt.style.use('ggplot')
import matplotlib
# Set GPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# Plotly 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
cd ..

/home/anirudh/github/aip-eval


In [3]:
from studio.evaluation.keras import metrics, utils, visualizer
from studio.evaluation.keras.evaluators import CNNEvaluator
from studio.evaluation.keras.evaluators import Evaluator
from studio.evaluation.keras.evaluators import SequentialCNNEvaluator
from studio.evaluation.keras.evaluators import VisualQAEvaluator
from visual_qa.visual_qa import VisualQA

Using TensorFlow backend.


In [4]:
report_dir = 'mini_evaluation/'
model_path = '/data/models/133_way_april_2019/ensemble/xception/model_max_acc_1gpu.h5'
visual_dictionary = '/home/anirudh/github/qa-data/data/visual_classifier/133_way_min_150_images_april_2019_model_map_diagnosis_nodes.json'
by_definition_csv = '/home/anirudh/github/qa-data/data/by-definition/custom_by_definition_katrina.csv'
qa_data_json = '/home/anirudh/github/qa-data/data/testset/survey_testset_derm_18july19.json'
valid_evidence = '/home/anirudh/github/qa-data/data/by-definition/valid_evidence.json'
visual_data_dir = '/data/datasets/visual_qa/new_cases'

In [5]:
cnn_evaluator = CNNEvaluator(
        concept_dictionary_path=None,
        custom_objects=None,
        concepts=None,
        model_path=model_path,
        batch_size=32,
        verbose=0)

Instructions for updating:
Colocations handled automatically by placer.


/home/anirudh/github/aip-eval/.venv/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning:

No training configuration found in save file: the model was *not* compiled. Compile it manually.



In [6]:
visual_qa_evaluator = VisualQAEvaluator(
    report_dir=report_dir,
    qa_data_json = qa_data_json,
    visual_dictionary = visual_dictionary,
    by_definition_csv = by_definition_csv,
    valid_evidence = valid_evidence
)

In [7]:
filtered_diagnosis_ids = ['AIP:0022217',
 'AIP:0000942',
 'AIP:0001341',
 'AIP:0100003',
 'AIP:0002471',
 'AIP:0000461',
 'AIP:0001398',
 'AIP:0100004',
 'AIP:0003513',
 'AIP:0002367',
 'AIP:0000122',
 'AIP:0000970',
 'AIP:0000506',
 'AIP:0003528',
 'AIP:0100060']

In [8]:
filtered_testset = [case for case in visual_qa_evaluator.filtered_qa_data if case['diagnosis_id'] in filtered_diagnosis_ids]

In [9]:
len(filtered_testset)

350

In [10]:
main_case = set()
for case in filtered_testset:
    main_case.add(case['case_id'].split('_')[0])

In [11]:
len(main_case)

218

In [12]:
visual_diagnosis_ids = list(visual_qa_evaluator.visual_qa.visual_diagnosis_ids)
labels = []
image_paths = []
for sample in filtered_testset:
    image_paths.append(os.path.join(visual_data_dir, sample["image_id"]))
    labels.append(visual_diagnosis_ids.index(sample['diagnosis_id']))
    
probs = cnn_evaluator.predict(image_paths)
labels = np.array(labels)

Making predictions from model  0
 9/10 [==========================>...] - ETA: 5s 
Processing remainder: 30


In [13]:
visual_result = cnn_evaluator.get_metrics(probs, labels, top_k=5, concept_labels=visual_qa_evaluator.visual_qa.visual_diagnosis_ids)

In [14]:
cnn_evaluator.plot_top_k_accuracy()

In [15]:
cnn_evaluator.show_results(mode='average')

,id,accuracy_top_1,accuracy_top_2,accuracy_top_3,accuracy_top_4,accuracy_top_5,weighted_precision,sensitivity,precision,f1_score,number_of_samples,number_of_classes
0,model_max_acc_1gpu.h5,0.411,0.534,0.6,0.669,0.709,0.753,0.039,0.072,0.463,350,133


In [16]:
visual_qa_evaluator.filtered_qa_data = filtered_testset

In [17]:
combination_result = visual_qa_evaluator.evaluate(probs, differential=True)

In [18]:
utils.compare_visual_by_definition_results(visual_result['average']['accuracy'] ,combination_result['average']['accuracy'] )

,Mode,top_1,top_2,top_3,top_4,top_5
0,Visual,0.411429,0.534286,0.600000,0.668571,0.708571
1,Visual_QA,0.317143,0.362857,0.391429,0.411429,0.437143
